# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [16]:
import pandas as pd 
from pyspark.sql import SparkSession

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [17]:
spark = SparkSession.builder.appName("tareaMajo").getOrCreate()

### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [18]:
df= spark.read.csv('datosTarea.csv', header=True, inferSchema=True)
df.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|          

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [19]:
from pyspark.sql.functions import col

df2 = df.filter(
    (col('Name').startswith('M')) & (col('Number of employees').between(4000, 7000))
)

res= df2.select('Name', 'Number of employees')

res.show()

+--------------------+-------------------+
|                Name|Number of employees|
+--------------------+-------------------+
|Mckinney, Riley a...|               4952|
|       Mcintosh-Mora|               4389|
|     Mckenzie-Melton|               4589|
|          Massey LLC|               5004|
|        Mays-Preston|               5786|
+--------------------+-------------------+



>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [20]:
df3 = df.filter(
    (~col('Country').startswith('b')) & (~col('Country').startswith('s')) & (~col('Country').startswith('m')) & (col('Networth') <= 500000)
)

res= df3.select('Name', 'Country', 'Networth')
res.show()

+--------------------+--------------------+--------+
|                Name|             Country|Networth|
+--------------------+--------------------+--------+
|         Ferrell LLC|    Papua New Guinea|  402269|
|      Holder-Sellers|        Turkmenistan|  105914|
|Keller, Campos an...|             Liberia|  329130|
|      Pacheco-Spears|              Sweden|   88435|
|         Harrell LLC|          Guadeloupe|  251274|
|         Jenkins Inc|        South Africa|  139725|
|Dickson, Richmond...|      Czech Republic|  359030|
|        Prince-David|    Christmas Island|  120289|
|         Rivas Group|           Australia|  477824|
|Sloan, Mays and W...|                Chad|   41975|
|Glass, Barrera an...|     Kyrgyz Republic|  300150|
|          Pineda-Cox|             Bolivia|  150880|
|Baker, Mccann and...|               Kenya|  188370|
|            Hahn PLC|             Belarus|  427224|
|Valentine, Fergus...|              Jersey|  412274|
|           Walls LLC|          Cape Verde|  1

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [21]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql import types as T

@pandas_udf(T.DoubleType())
def ajuste(profit):
    return profit - profit.mean()

dfAjuste = df.withColumn('profits_adjusted', ajuste(df['Profit']))
dfAjuste = dfAjuste.select('Name', 'profits_adjusted')
dfAjuste.show()

+--------------------+----------------+
|                Name|profits_adjusted|
+--------------------+----------------+
|         Ferrell LLC|          -194.0|
|Mckinney, Riley a...|          -318.0|
|          Hester Ltd|          4802.0|
|      Holder-Sellers|         -4119.0|
|         Mayer Group|         -3720.0|
|      Henry-Thompson|         -6930.0|
|      Hansen-Everett|         -6619.0|
|       Mcintosh-Mora|         -6489.0|
|            Carr Inc|          3467.0|
|          Gaines Inc|          2516.0|
|          Kidd Group|          3105.0|
|        Crane-Clarke|         -4022.0|
|Keller, Campos an...|          4724.0|
|         Glover-Pope|          5490.0|
|      Pacheco-Spears|           358.0|
|         Hodge-Ayers|          3478.0|
|Bowers, Guerra an...|           211.0|
|     Mckenzie-Melton|          4639.0|
|         Branch-Mann|         -3574.0|
|      Weiss and Sons|          5760.0|
+--------------------+----------------+
only showing top 20 rows



### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [22]:
df.groupBy('Industry').max('Profit').show(3)


+--------------------+-----------+
|            Industry|max(Profit)|
+--------------------+-----------+
|Primary / Seconda...|      15424|
|     Broadcast Media|      12510|
|           Wholesale|       9754|
+--------------------+-----------+
only showing top 3 rows



>Agrupa por industry y calcula el promedio de empleados que tienen

In [28]:
print(df.columns)

df.groupBy('Industry').avg('Number of employees').show()


['Index', 'Organization Id', 'Name', 'Website', 'Country', 'Description', 'Founded', 'Industry', 'Number of employees', 'Networth', 'stock_price', 'profit']
+--------------------+------------------------+
|            Industry|avg(Number of employees)|
+--------------------+------------------------+
|Primary / Seconda...|       6457.666666666667|
|     Broadcast Media|                  2589.0|
|           Wholesale|                  5010.0|
|Investment Manage...|                  3133.5|
|    Food / Beverages|                  9011.0|
|  Gambling / Casinos|                  4873.0|
|Logistics / Procu...|                  4155.0|
|            Maritime|                   769.0|
|            Wireless|                  6146.0|
|Education Management|                   339.0|
|       Arts / Crafts|                  2800.0|
|           Insurance|                  1215.0|
|  Financial Services|                  5157.0|
|Business Supplies...|                  9097.0|
|Consumer Electronics|     

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [29]:
df.createOrReplaceTempView('companies')

res = spark.sql("""
SELECT COUNT(*) as count
FROM companies
WHERE Founded > 2000
""")

res.show()

+-----+
|count|
+-----+
|   38|
+-----+



### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [35]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

featureassembler=VectorAssembler(inputCols=["Number of employees","Networth", "stock_price"],outputCol="Independent Features")
output=featureassembler.transform(df)
res = output.select('Name', 'Number of employees', 'Networth', 'stock_price', 'Independent Features')
res.show()

data=output.select("Independent Features","Networth")

train,test=data.randomSplit([0.75, 0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Networth')
regressor=regressor.fit(train)

predictions = regressor.transform(test)

pRes=regressor.evaluate(test)

predicciones = pRes.predictions
predicciones.show()

+--------------------+-------------------+--------+-----------+--------------------+
|                Name|Number of employees|Networth|stock_price|Independent Features|
+--------------------+-------------------+--------+-----------+--------------------+
|         Ferrell LLC|               3498|  402269|         33|[3498.0,402269.0,...|
|Mckinney, Riley a...|               4952|  569480|         49|[4952.0,569480.0,...|
|          Hester Ltd|               5287|  608005|         26|[5287.0,608005.0,...|
|      Holder-Sellers|                921|  105914|         23|[921.0,105914.0,2...|
|         Mayer Group|               7870|  905049|         31|[7870.0,905049.0,...|
|      Henry-Thompson|               4914|  565110|         49|[4914.0,565110.0,...|
|      Hansen-Everett|               7832|  900679|         13|[7832.0,900679.0,...|
|       Mcintosh-Mora|               4389|  504734|         19|[4389.0,504734.0,...|
|            Carr Inc|               8167|  939204|         16|[8

23/12/10 16:58:04 WARN Instrumentation: [2c974255] regParam is zero, which might cause numerical instability and overfitting.


+--------------------+--------+----------+
|Independent Features|Networth|prediction|
+--------------------+--------+----------+
|[696.0,80040.0,33.0]|   80040|   80040.0|
|[769.0,88435.0,34.0]|   88435|   88435.0|
|[921.0,105914.0,2...|  105914|  105914.0|
|[1215.0,139725.0,...|  139725|  139725.0|
|[1825.0,209874.0,...|  209874|  209874.0|
|[2073.0,238394.0,...|  238394|  238394.0|
|[3450.0,396749.0,...|  396749|  396749.0|
|[3816.0,438839.0,...|  438839|  438839.0|
|[3934.0,452409.0,...|  452409|  452409.0|
|[4914.0,565110.0,...|  565110|  565110.0|
|[5786.0,665390.0,...|  665390|  665390.0|
|[6986.0,803389.0,...|  803389|  803389.0|
|[7034.0,808909.0,...|  808909|  808909.0|
|[7443.0,855944.0,...|  855944|  855944.0|
|[7529.0,865834.0,...|  865834|  865834.0|
|[7870.0,905049.0,...|  905049|  905049.0|
|[7961.0,915514.0,...|  915514|  915514.0|
|[8433.0,969794.0,...|  969794|  969794.0|
|[8445.0,971174.0,...|  971174|  971174.0|
|[8480.0,975199.0,...|  975199|  975199.0|
+----------

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [36]:
predicciones.toPandas().to_csv('predicciones.csv', index = False)